In [4]:
pip install torch torchvision pillow opencv-python
!pip install flask
!pip install flask_cors
!pip install opencv-python
!pip install opencv-python-headless

   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/159.8 MB 6.8 MB/s eta 0:00:24
   ---------------------------------------- 0.2/159.8 MB 2.6 MB/s eta 0:01:01
   ---------------------------------------- 0.3/159.8 MB 1.8 MB/s eta 0:01:31
   ---------------------------------------- 0.3/159.8 MB 2.0 MB/s eta 0:01:21
   ---------------------------------------- 0.6/159.8 MB 2.3 MB/s eta 0:01:09
   ---------------------------------------- 0.9/159.8 MB 3.2 MB/s eta 0:00:50
   ---------------------------------------- 1.3/159.8 MB 3.9 MB/s eta 0:00:41
   ---------------------------------------- 1.8/159.8 MB 4.7 MB/s eta 0:00:34
    --------------------------------------- 2.2/159.8 MB 5.4 MB/s eta 0:00:30
    --------------------------------------- 2.3/159.8 MB 5.2 MB/s eta 0:00:31
    --------------------------------------- 2.3/159.8 MB 4.5 MB/s eta 0:00:36
    --------------------------------------- 2.4/159.8 MB 4.5 MB/s eta 0

In [1]:
import uuid
import os
from flask import Flask, request, jsonify, session
from flask_cors import CORS, cross_origin
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json
import torch
import torchvision.transforms as T
from torchvision.models.detection import maskrcnn_resnet50_fpn
from PIL import Image

In [1]:
#########################################################################################
#                                   각종 함수 선언                                       #  
#########################################################################################

# HSV 색상을 기준으로 색상 분류 함수
def classify_color_by_hsv(hsv):
    h, s, v = hsv
    if (0 <= h <= 10 or 340 <= h <= 360) and (30 <= s <= 100) and (50 <= v <= 100):
        return '레드'
    elif (10 <= h <= 25) and (30 <= s <= 100) and (50 <= v <= 100):
        return '오렌지'
    elif (47 <= h <= 62) and (30 <= s <= 100) and (50 <= v <= 100):
        return '옐로우'
    elif (65 <= h <= 158) and (30 <= s <= 100) and (50 <= v <= 100):
        return '그린'
    elif (169 <= h <= 193) and (0 <= s <= 100) and (50 <= v <= 100):
        return '라이트블루'
    elif (195 <= h <= 255) and (30 <= s <= 100) and (50 <= v <= 100):
        return '네이비'
    elif (271 <= h <= 300) and (0 <= s <= 100) and (50 <= v <= 100):
        return '퍼플'
    elif (313 <= h <= 323) and (30 <= s <= 100) and (50 <= v <= 100):
        return '핑크'
    elif (12 <= h <= 47) and (10 <= s <= 77) and (70 <= v <= 100):
        return '베이지'
    elif (0 <= h <= 16) and (20 <= s <= 30) and (20 <= v <= 85):
        return '브라운'
    elif (0 <= h <= 360) and (0 <= s <= 20) and (20 <= v <= 95):
        return '그레이'
    elif (0 <= h <= 360) and (0 <= s <= 10) and (95 <= v <= 100):
        return '화이트'
    elif (0 <= h <= 360) and (0 <= s <= 100) and (0 <= v <= 20):
        return '블랙'
    else:
        min_distance = float('inf')
        closest_color = 'unknown'
        color_ranges = [
            ('레드', (0, 50, 50), (10, 100, 100)),
            ('레드2', (340, 50, 50), (360, 100, 100)),
            ('오렌지', (10, 50, 50), (25, 100, 100)),
            ('옐로우', (47, 50, 50), (62, 100, 100)),
            ('그린', (65, 30, 50), (158, 100, 100)),
            ('라이트블루', (169, 50, 50), (193, 100, 100)),
            ('네이비', (195, 50, 50), (255, 100, 100)),
            ('퍼플', (271, 50, 50), (300, 100, 100)),
            ('핑크', (313, 50, 50), (323, 100, 100)),
            ('베이지', (12, 10, 77), (47, 100, 100)),
            ('브라운', (0, 30, 20), (16, 55, 85)),
            ('그레이', (0, 0, 20), (360, 10, 95)),
            ('화이트', (0, 0, 95), (360, 5, 100)),
            ('블랙', (0, 0, 0), (360, 100, 20))
        ]
        
        for color, lower, upper in color_ranges:
            h_mean = (lower[0] + upper[0]) / 2
            s_mean = (lower[1] + upper[1]) / 2
            v_mean = (lower[2] + upper[2]) / 2
            distance = np.sqrt((h - h_mean)**2 + (s - s_mean)**2 + (v - v_mean)**2)
            if distance < min_distance:
                min_distance = distance
                closest_color = color
        
        return closest_color

# 객체 영역 제거 및 배경 채우기 함수
def remove_objects_and_fill_background(image_path, model, score_threshold=0.7):
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)
    
    with torch.no_grad():
        predictions = model(img_tensor)
    
    masks = predictions[0]['masks'].squeeze().detach().cpu().numpy()
    scores = predictions[0]['scores'].detach().cpu().numpy()
    
    background_mask = np.ones_like(masks[0], dtype=bool) if len(masks) > 0 else None
    
    if background_mask is not None:
        for mask, score in zip(masks, scores):
            if score >= score_threshold:
                background_mask = np.logical_and(background_mask, ~mask.astype(bool))
    
    bg_color = None
    if background_mask is not None:
        try:
            bg_color = extract_background_color(img_np, background_mask)
        except ValueError as e:
            print(f'배경 색상을 추출할 수 없습니다: {e}')
    
    filled_image = img_np.copy()
    if bg_color is not None:
        filled_image[~background_mask] = bg_color
    
    return filled_image

# 배경 색상 추출 함수
def extract_background_color(image, background_mask):
    if background_mask is None:
        raise ValueError("배경 마스크가 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_pixels = image[background_mask]
    
    if len(bg_pixels) == 0:
        raise ValueError("배경 픽셀이 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_color = np.mean(bg_pixels, axis=0, dtype=int)
    return bg_color

# 주요 색상 찾기 함수 (KMeans 사용)
def find_dominant_color(image_rgb, k=10):  # 클러스터 수를 10으로 증가
    pixels = image_rgb.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    label_counts = np.bincount(labels)
    dominant_color = centers[np.argmax(label_counts)]
    cluster_ratios = label_counts / len(labels)
    return dominant_color.astype(int), centers.astype(int), cluster_ratios

# 주요 색상을 시각적으로 확인하는 함수
def plot_colors(centers, ratios):
    patches = np.zeros((50, 300, 3), dtype=int)
    start = 0
    for center, ratio in zip(centers, ratios):
        end = start + int(300 * ratio)
        patches[:, start:end, :] = center
        start = end
    return patches

# RGB에서 HSV로 변환하는 함수
def convert_rgb_to_hsv(rgb_color):
    rgb_color = np.uint8([[rgb_color]])
    hsv_color = cv2.cvtColor(rgb_color, cv2.COLOR_RGB2HSV)
    return hsv_color[0][0]  # H, S, V 값은 0~180, 0~255, 0~255 범위

# HSV 값을 0~360, 0~100, 0~100 범위로 변환하는 함수
def normalize_hsv(hsv):
    h, s, v = hsv
    h = h * 2  # 0~180 -> 0~360
    s = s / 255 * 100  # 0~255 -> 0~100
    v = v / 255 * 100  # 0~255 -> 0~100
    return [h, s, v]

# 톤 결정 함수
def determine_tone(hsv):
    tone_classes = {
        '비비드': (75, 75),
        '스트롱': (60, 67.5),
        '딥': (50, 37.5),
        '라이트': (37.5, 87.5),
        '소프트': (37.5, 62.5),
        '페일': (12.5, 87.5),
        '다크': (62.5, 12.5),
        '덜': (12.5, 37.5)
    }
    
    h, s, v = hsv
    min_distance = float('inf')
    closest_tone = '기타'
    
    for tone, (s_mean, v_mean) in tone_classes.items():
        distance = np.sqrt((s - s_mean)**2 + (v - v_mean)**2)
        if distance < min_distance:
            min_distance = distance
            closest_tone = tone
    
    return closest_tone

In [ ]:
#########################################################################################
#                                  install & import                                     #  
#########################################################################################




# 모델 로드 (미리 학습된 Mask R-CNN)
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

app = Flask(__name__)
CORS(app, supports_credentials=True) # 자격 증명(쿠키, 인증 헤더 등)이 포함된 요청을 허용
app.secret_key = 'AgainE_Flask'

UPLOAD_FOLDER = 'data/FlaskServer/'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

TOMCAT_UPLOAD_URL = 'http://localhost:8085/AgainE_Project/user_room'  # 톰캣 서버의 파일 저장 엔드포인트



#########################################################################################
#                         URL Mapping & 서버 실행될 때 함수                               #  
#########################################################################################

@app.route("/flaskServer", methods=["POST"])
@cross_origin(origins='http://localhost:8085', supports_credentials=True)  # 클라이언트의 도메인에 맞게 origins 수정
def flaskServer():
    print("서버 요청 들어옴")
    if 'imageFile' not in request.files:
        return jsonify({'error': '파일이 없습니다'}), 400

    file = request.files['imageFile']
    if file.filename == '':
        return jsonify({'error': '파일 이름이 없습니다'}), 400

    if file:
        # UUID 생성
        unique_id = str(uuid.uuid4())[:8]  # UUID의 앞 8자리만 사용 (예: a1b2c3d4)
        # 파일 확장자 추출
        file_extension = os.path.splitext(file.filename)[1].lower()  # 소문자로 변환하여 확장자 추출
        # 새 파일명 생성
        filename = f"{unique_id}{file_extension}"    # 새로운 파일명 (예: a1b2c3d4.png)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)   # 파이썬 로컬 경로 (예 : data/flaskTest/a1b2c3d4.png)
        file.save(filepath)  # 해당 경로에 파일 저장

        #################################################   색 분류 ################################
        filled_image = remove_objects_and_fill_background(filepath, model)

        # 전처리된 이미지에서 주요 색상 및 톤 분석
        dominant_color, centers, cluster_ratios = find_dominant_color(filled_image)  # 전처리된 이미지에서 주요 색상 찾기
        dominant_color_hsv = convert_rgb_to_hsv(dominant_color)
        normalized_hsv = normalize_hsv(dominant_color_hsv)

        tone = determine_tone(normalized_hsv)
        color = classify_color_by_hsv(normalized_hsv)  # 첫 번째 HSV 값을 전달
    
        if color == "레드" or color == "레드2" :
            color = "레드"
        
        print(f'색상: {color}, 톤: {tone}')
    
        #########################################################################################
        #                       세션 정보 불러오기 & 저장 / 톰캣한테 보내기                      #  
        #########################################################################################
            
        loginUserJson = request.form.get('loginUserJson')
        app.logger.debug(f'Received loginUserJson: {loginUserJson}')  # 디버그 로그에 출력
        
        if loginUserJson:
            try:
                user = json.loads(loginUserJson)
                session['login_user'] = user  # 세션에 로그인한 사용자의 userDTO 저장

                # 톰캣 서버(servlet)로 파일 전송
                with open(filepath, 'rb') as f:
                    files = {'file': (filename, f)}
                    data = {'color': color, 'tone' : tone, 'login_user': json.dumps(user)}
                    response = requests.post(TOMCAT_UPLOAD_URL, files=files, data=data)
                    
                    if response.status_code != 200 :
                        return jsonify({"error": "Failed to send data to Tomcat server"}), 500
                        
            except json.JSONDecodeError as e:
                print(f'JSONDecodeError: {e}')  # print 사용
                return jsonify({"error": "Invalid JSON format"}), 400
        else:
            return jsonify({"error": "No userDTO provided"}), 400

        #########################################################################################
        #                                ajax 응답 데이터                                        #  
        #########################################################################################
        
        return jsonify(response.json()), 200

    return jsonify({'error': '파일을 처리하는 중 오류가 발생했습니다'}), 500

#########################################################################################
#                                         서버실행                                       #  
#########################################################################################
if __name__ == "__main__":
    app.run(host='192.168.219.200', port=5058, debug=True, use_reloader=False)


C:\Users\SMHRD\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://192.168.219.200:5058
Press CTRL+C to quit


서버 요청 들어옴


[2024-07-02 14:45:25,561] DEBUG in 1520568998: Received loginUserJson: {"user_id":"tester","user_pw":"7110eda4d09e062aa5e4a390b0a572ac0d2c0220","user_name":"테스터","user_email":"tester@test.com","user_phone":"01011111111","user_addr":"테스트","user_room_tone":"페일","user_room_color":"그레이","user_room_url":"29b5ea5f.png","joined_at":[2024,6,24,14,37,53],"user_type":"t"}


색상: 그레이, 톤: 페일


192.168.219.200 - - [02/Jul/2024 14:45:25] "POST /flaskServer HTTP/1.1" 200 -


# =============

#   리얼찐찐막 모델

# =============

In [ ]:
#########################################################################################
#                                  install & import                                     #  
#########################################################################################
!pip install flask
!pip install flask_cors
!pip install opencv-python
!pip install opencv-python-headless

import uuid
import os
from flask import Flask, request, jsonify, session
from flask_cors import CORS, cross_origin
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json
import torch
import torchvision.transforms as T
from torchvision.models.detection import maskrcnn_resnet50_fpn
from PIL import Image
import tarfile

# 모델 로드 (미리 학습된 Mask R-CNN)
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

app = Flask(__name__)
CORS(app, supports_credentials=True) # 자격 증명(쿠키, 인증 헤더 등)이 포함된 요청을 허용
app.secret_key = 'AgainE_Flask'

UPLOAD_FOLDER = 'data/FlaskServer/'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

TOMCAT_UPLOAD_URL = 'http://localhost:8085/AgainE_Project/user_room'  # 톰캣 서버의 파일 저장 엔드포인트

#########################################################################################
#                                   각종 함수 선언                                       #  
#########################################################################################

# HSV 색상을 기준으로 색상 분류 함수
def classify_color_by_hsv(hsv):
    h, s, v = hsv
    
    if (0 <= h <= 10 or 340 <= h <= 360) and (30 <= s <= 100) and (50 <= v <= 100):
        return '레드'
    elif (10 <= h <= 25) and (30 <= s <= 100) and (50 <= v <= 100):
        return '오렌지'
    elif (47 <= h <= 62) and (30 <= s <= 100) and (50 <= v <= 100):
        return '옐로우'
    elif (65 <= h <= 158) and (30 <= s <= 100) and (50 <= v <= 100):
        return '그린'
    elif (169 <= h <= 193) and (0 <= s <= 100) and (50 <= v <= 100):
        return '라이트블루'
    elif (195 <= h <= 255) and (30 <= s <= 100) and (50 <= v <= 100):
        return '네이비'
    elif (271 <= h <= 300) and (0 <= s <= 100) and (50 <= v <= 100):
        return '퍼플'
    elif (313 <= h <= 323) and (30 <= s <= 100) and (50 <= v <= 100):
        return '핑크'
    elif (12 <= h <= 47) and (10 <= s <= 77) and (70 <= v <= 100):
        return '베이지'
    elif (0 <= h <= 16) and (20 <= s <= 30) and (20 <= v <= 85):
        return '브라운'
    elif (0 <= h <= 360) and (0 <= s <= 20) and (20 <= v <= 95):
        return '그레이'
    elif (0 <= h <= 360) and (0 <= s <= 10) and (95 <= v <= 100):
        return '화이트'
    elif (0 <= h <= 360) and (0 <= s <= 100) and (0 <= v <= 20):
        return '블랙'
    else:
        min_distance = float('inf')
        closest_color = 'unknown'
        # color_ranges = [
        #     ('레드', (0, 50, 50), (10, 100, 100)),
        #     ('레드2', (340, 50, 50), (360, 100, 100)),
        #     ('오렌지', (10, 50, 50), (25, 100, 100)),
        #     ('옐로우', (47, 50, 50), (62, 100, 100)),
        #     ('그린', (65, 30, 50), (158, 100, 100)),
        #     ('라이트블루', (169, 50, 50), (193, 100, 100)),
        #     ('네이비', (195, 50, 50), (255, 100, 100)),
        #     ('퍼플', (271, 50, 50), (300, 100, 100)),
        #     ('핑크', (313, 50, 50), (323, 100, 100)),
        #     ('베이지', (12, 10, 77), (47, 100, 100)),
        #     ('브라운', (0, 30, 20), (16, 55, 85)),
        #     ('그레이', (0, 0, 20), (360, 10, 95)),
        #     ('화이트', (0, 0, 95), (360, 5, 100)),
        #     ('블랙', (0, 0, 0), (360, 100, 20))
        # ]

        color_ranges = {
            '레드': ((0, 50, 50), (10, 100, 100)),
            '레드2': ((340, 50, 50), (360, 100, 100)),
            '오렌지': ((10, 50, 50), (25, 100, 100)),
            '옐로우': ((47, 50, 50), (62, 100, 100)),
            '그린': ((65, 30, 50), (158, 100, 100)),
            '라이트블루': ((169, 50, 50), (193, 100, 100)),
            '네이비': ((195, 50, 50), (255, 100, 100)),
            '퍼플': ((271, 50, 50), (300, 100, 100)),
            '핑크': ((313, 50, 50), (323, 100, 100)),
            '베이지': ((12, 10, 77), (47, 100, 100)),
            '브라운': ((0, 30, 20), (16, 55, 85)),
            '그레이': ((0, 0, 20), (360, 10, 95)),
            '화이트': ((0, 0, 95), (360, 5, 100)),
            '블랙': ((0, 0, 0), (360, 100, 20))
        }
        
        # for color, (h_min, s_min, v_min, h_max, s_max, v_max) in color_ranges.items():
        for color, ((h_min, s_min, v_min), (h_max, s_max, v_max)) in color_ranges.items():
            h_mean = (h_min + h_max) / 2
            s_mean = (s_min + s_max) / 2
            v_mean = (v_min + v_max) / 2
            distance = np.sqrt((h - h_mean)**2 + (s - s_mean)**2 + (v - v_mean)**2)
            if distance < min_distance:
                min_distance = distance
                closest_color = color
        
        return closest_color

# 모델 및 라이브러리 로드
def load_model(weights):
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

# 이미지를 로드하고 전처리
def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or invalid image path: {image_path}")
    return image

# YOLOv5로 객체 탐지
def detect_objects(model, image):
    results = model(image)
    return results.pandas().xyxy[0]

# 최빈색 계산
def get_most_frequent_color(image):
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=1, n_init=10, random_state=0)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_.astype(int)[0]
    return dominant_color

# 객체 영역 제거 및 배경 채우기 함수
def remove_objects_and_fill_background(image, detections):
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255
    for _, row in detections.iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        mask[y1:y2, x1:x2] = 0
    
    # 최빈색 계산
    filled_image = image.copy()
    dominant_color = get_most_frequent_color(image)
    
    # 마스크를 사용하여 최빈색으로 채우기
    filled_image[mask == 0] = dominant_color
    
    return filled_image


# 배경 색상 추출 함수
def extract_background_color(image, background_mask):
    """
        주어진 이미지와 배경 마스크를 사용하여 배경 색상을 추출하는 함수
        
        Args:
        - image (numpy.ndarray): 입력 이미지 배열 (RGB)
        - background_mask (numpy.ndarray): 배경을 나타내는 이진 마스크 배열
        
        Returns:
        - bg_color (numpy.ndarray): 추출된 배경 색상 값 (RGB)
        
        Raises:
        - ValueError: 배경 마스크가 없거나 배경 픽셀이 없는 경우 발생
    """
    if background_mask is None:
        raise ValueError("배경 마스크가 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_pixels = image[background_mask]                           # 원본에 배경 마스크를 적용해서 bg_pixels 생성
    
    if len(bg_pixels) == 0:
        raise ValueError("배경 픽셀이 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_color = np.mean(bg_pixels, axis=0, dtype=int)             # bg_pixels의 평균 값을 bg_color로 생성
    return bg_color # 1. 원본 + 마스크 = bg_pixels >> 2. bg_pixels / 평균값 = bg_color >> 3. 최종적으로 최빈 색상으로 덮어서 출력

# 주요 색상 찾기 함수 (KMeans 사용)
def find_dominant_color(image_rgb, k=10):
    pixels = image_rgb.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    label_counts = np.bincount(labels)
    dominant_color = centers[np.argmax(label_counts)]
    return dominant_color.astype(int)
# 주요 색상을 시각적으로 확인하는 함수
def plot_colors(centers, ratios):
    """
      주요 색상을 시각적으로 확인할 수 있는 이미지 패치를 생성하는 함수
      
      Args:
      - centers (numpy.ndarray): 클러스터 중심 색상 배열
      - ratios (numpy.ndarray): 클러스터 비율 배열
      
      Returns:
      - patches (numpy.ndarray): 시각적 확인을 위한 이미지 패치 배열
    """
    patches = np.zeros((50, 300, 3), dtype=int)
    start = 0
    for center, ratio in zip(centers, ratios):
        end = start + int(300 * ratio)
        patches[:, start:end, :] = center
        start = end
    return patches

# RGB에서 HSV로 변환하는 함수
def convert_rgb_to_hsv(rgb_color):
    """
        주어진 RGB 색상을 HSV 형식으로 변환하는 함수
        
        Args:
        - rgb_color (numpy.ndarray): RGB 색상 값 배열
        
        Returns:
        - hsv_color (numpy.ndarray): 변환된 HSV 색상 값 배열
    """
    rgb_color = np.uint8([[rgb_color]])
    hsv_color = cv2.cvtColor(rgb_color, cv2.COLOR_RGB2HSV)
    return hsv_color[0][0]  # H, S, V 값은 0~180, 0~255, 0~255 범위

# HSV 값을 0~360, 0~100, 0~100 범위로 변환하는 함수
def normalize_hsv(hsv):
    h, s, v = hsv
    h = h * 2  # 0~180 -> 0~360
    s = s / 255 * 100  # 0~255 -> 0~100
    v = v / 255 * 100  # 0~255 -> 0~100
    return [h, s, v]


# 톤 결정 함수
def determine_tone(hsv):
    tone_classes = {
        '비비드': (75, 75),
        '스트롱': (60, 67.5),
        '딥': (50, 37.5),
        '라이트': (37.5, 87.5),
        '소프트': (37.5, 62.5),
        '페일': (12.5, 87.5),
        '다크': (62.5, 12.5),
        '덜': (12.5, 37.5)
    }
    
    h, s, v = hsv
    min_distance = float('inf')
    closest_tone = '기타'
    
    for tone, (s_mean, v_mean) in tone_classes.items():
        distance = np.sqrt((s - s_mean)**2 + (v - v_mean)**2)
        if distance < min_distance:
            min_distance = distance
            closest_tone = tone
    
    return closest_tone


# 사진 해상도 조절 및 numpy배열로 변경
def resize_image(image, size):
    img = Image.open(image)
    img.thumbnail(size, Image.Resampling.LANCZOS)
    image_np = np.array(img)
    return image_np

#########################################################################################
#                         URL Mapping & 서버 실행될 때 함수                               #  
#########################################################################################

@app.route("/flaskServer", methods=["POST"])
@cross_origin(origins='http://localhost:8085', supports_credentials=True)  # 클라이언트의 도메인에 맞게 origins 수정
def flaskServer():
    print("서버 요청 들어옴")

    ################################# 서버에 이미지 저장 ################################
    
    if 'imageFile' not in request.files:
        return jsonify({'error': '파일이 없습니다'}), 400

    file = request.files['imageFile']
    if file.filename == '':
        return jsonify({'error': '파일 이름이 없습니다'}), 400

    if file:
        # UUID 생성
        unique_id = str(uuid.uuid4())[:8]  # UUID의 앞 8자리만 사용 (예: a1b2c3d4)
        # 파일 확장자 추출
        file_extension = os.path.splitext(file.filename)[1].lower()  # 소문자로 변환하여 확장자 추출
        # 새 파일명 생성
        filename = f"{unique_id}{file_extension}"    # 새로운 파일명 (예: a1b2c3d4.png)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)   # 파이썬 로컬 경로 (예 : data/flaskTest/a1b2c3d4.png)
        file.save(filepath)  # 해당 경로에 파일 저장

        ############################## 이미지 불러오기 & 전처리 ################################
        
        model = load_model('yolov5s.pt')
        # image = load_image(filepath)
        image = resize_image(filepath, (800, 600))  # 적절한 해상도로 리사이즈
    
        # detections = detect_objects(model, filepath)
        detections = detect_objects(model, image)
    
        filled_image = remove_objects_and_fill_background(image, detections)

        #################################### 색 분류  #####################################
        
        dominant_color_rgb = find_dominant_color(filled_image)
        dominant_color_hsv = convert_rgb_to_hsv(dominant_color_rgb)
        normalized_dominant_color_hsv = normalize_hsv(dominant_color_hsv)
        color = classify_color_by_hsv(normalized_dominant_color_hsv)
        if color == "레드" or color == "레드2" :
            color = "레드"
        
        tone = determine_tone(normalized_dominant_color_hsv)

        print(f'색상: {color}, 톤: {tone}')
    
        #########################################################################################
        #                       세션 정보 불러오기 & 저장 / 톰캣한테 보내기                      #  
        #########################################################################################
            
        loginUserJson = request.form.get('loginUserJson')
        app.logger.debug(f'Received loginUserJson: {loginUserJson}')  # 디버그 로그에 출력
        
        if loginUserJson:
            try:
                user = json.loads(loginUserJson)
                session['login_user'] = user  # 세션에 로그인한 사용자의 userDTO 저장

                # 톰캣 서버(servlet)로 파일 전송
                with open(filepath, 'rb') as f:
                    files = {'file': (filename, f)}
                    data = {'color': color, 'tone' : tone, 'login_user': json.dumps(user)}
                    response = requests.post(TOMCAT_UPLOAD_URL, files=files, data=data)
                    
                    if response.status_code != 200 :
                        return jsonify({"error": "Failed to send data to Tomcat server"}), 500
                        
            except json.JSONDecodeError as e:
                print(f'JSONDecodeError: {e}')  # print 사용
                return jsonify({"error": "Invalid JSON format"}), 400
        else:
            return jsonify({"error": "No userDTO provided"}), 400

        #########################################################################################
        #                                ajax 응답 데이터                                        #  
        #########################################################################################
        
        return jsonify(response.json()), 200

    return jsonify({'error': '파일을 처리하는 중 오류가 발생했습니다'}), 500

#########################################################################################
#                                         서버실행                                       #  
#########################################################################################
if __name__ == "__main__":
    app.run(host='192.168.219.200', port=5058, debug=True, use_reloader=False)


C:\Users\SMHRD\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SMHRD\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://192.168.219.200:5058
Press CTRL+C to quit


서버 요청 들어옴


Using cache found in C:\Users\SMHRD/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-31 Python-3.11.7 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
[2024-08-06 18:04:31,703] DEBUG in 2692736458: Received loginUserJson: {"user_id":"tester","user_pw":"7110eda4d09e062aa5e4a390b0a572ac0d2c0220","user_name":"테스터","user_email":"tester@test.com","user_phone":"01011111111","user_addr":"테스트","user_room_tone":"페일","user_room_color":"그레이","user_room_url":"015621f7.jpg","joined_at":[2024,6,24,14,37,53],"user_type":"t"}


...

192.168.219.200 - - [06/Aug/2024 18:04:32] "POST /flaskServer HTTP/1.1" 200 -


서버 요청 들어옴


Using cache found in C:\Users\SMHRD/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-31 Python-3.11.7 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
[2024-08-06 18:08:20,632] DEBUG in 2692736458: Received loginUserJson: {"user_id":"tester1111","user_pw":"7110eda4d09e062aa5e4a390b0a572ac0d2c0220","user_name":"테스터","user_email":"testtest1111@gmail.com","user_phone":"010111111111","user_addr":"(61481) 광주 동구 중앙로 185-1 광주빌딩  (금남로4가)","user_room_tone":null,"user_room_color":null,"user_room_url":null,"joined_at":[2024,8,6,18,7,38],"user_type":"f"}


색상: 브라운, 톤: 소프트


192.168.219.200 - - [06/Aug/2024 18:08:21] "POST /flaskServer HTTP/1.1" 200 -


In [4]:
!pip install --upgrade backports setuptools

ERROR: Could not find a version that satisfies the requirement backports (from versions: none)
ERROR: No matching distribution found for backports


In [5]:
!pip install --upgrade setuptools

In [6]:
!pip install --upgrade jaraco.context

In [7]:
!pip install backports.tarfile

In [1]:
#########################################################################################
#                                  install & import                                     #  
#########################################################################################
!pip install flask
!pip install flask_cors
!pip install opencv-python
!pip install opencv-python-headless

import uuid
import os
from flask import Flask, request, jsonify, session
from flask_cors import CORS, cross_origin
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json
import torch
import torchvision.transforms as T
from PIL import Image
from io import BytesIO

app = Flask(__name__)
CORS(app, supports_credentials=True) # 자격 증명(쿠키, 인증 헤더 등)이 포함된 요청을 허용
app.secret_key = 'AgainE_Flask'

TOMCAT_UPLOAD_URL = 'http://localhost:8085/AgainE_Project/user_room'  # 톰캣 서버의 파일 저장 엔드포인트

#########################################################################################
#                                   각종 함수 선언                                       #  
#########################################################################################

# HSV 색상을 기준으로 색상 분류 함수
def classify_color_by_hsv(hsv):
    h, s, v = hsv
    
    if (0 <= h <= 10 or 340 <= h <= 360) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'red'
    elif (10 <= h <= 25) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'orange'
    elif (47 <= h <= 62) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'yellow'
    elif (65 <= h <= 158) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'green'
    elif (169 <= h <= 193) and (0 <= s <= 100) and (50 <= v <= 100):
        return 'light_blue'
    elif (195 <= h <= 255) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'navy'
    elif (271 <= h <= 300) and (0 <= s <= 100) and (50 <= v <= 100):
        return 'purple'
    elif (313 <= h <= 323) and (30 <= s <= 100) and (50 <= v <= 100):
        return 'pink'
    elif (12 <= h <= 47) and (10 <= s <= 77) and (70 <= v <= 100):
        return 'beige'
    elif (0 <= h <= 16) and (20 <= s <= 30) and (20 <= v <= 85):
        return 'brown'
    elif (0 <= h <= 360) and (0 <= s <= 20) and (20 <= v <= 95):
        return 'gray'
    elif (0 <= h <= 360) and (0 <= s <= 10) and (95 <= v <= 100):
        return 'white'
    elif (0 <= h <= 360) and (0 <= s <= 100) and (0 <= v <= 20):
        return 'black'
    else:
        min_distance = float('inf')
        closest_color = 'unknown'
        color_ranges = [
            ('red', (0, 50, 50), (10, 100, 100)),
            ('red2', (340, 50, 50), (360, 100, 100)),
            ('orange', (10, 50, 50), (25, 100, 100)),
            ('yellow', (47, 50, 50), (62, 100, 100)),
            ('green', (65, 30, 50), (158, 100, 100)),
            ('light_blue', (169, 50, 50), (193, 100, 100)),
            ('navy', (195, 50, 50), (255, 100, 100)),
            ('purple', (271, 50, 50), (300, 100, 100)),
            ('pink', (313, 50, 50), (323, 100, 100)),
            ('beige', (12, 10, 77), (47, 100, 100)),
            ('brown', (0, 30, 20), (16, 55, 85)),
            ('gray', (0, 0, 20), (360, 10, 95)),
            ('white', (0, 0, 95), (360, 5, 100)),
            ('black', (0, 0, 0), (360, 100, 20))
        ]
        
        for color, (h_min, s_min, v_min, h_max, s_max, v_max) in color_ranges:
            h_mean = (h_min + h_max) / 2
            s_mean = (s_min + s_max) / 2
            v_mean = (v_min + v_max) / 2
            distance = np.sqrt((h - h_mean)**2 + (s - s_mean)**2 + (v - v_mean)**2)
            if distance < min_distance:
                min_distance = distance
                closest_color = color
        
        return closest_color

# 모델 및 라이브러리 로드
def load_model(weights='yolov5s.pt'):
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    return model

# 이미지를 메모리에서 로드하고 전처리
def load_image_from_bytes(image_bytes):
    image = Image.open(BytesIO(image_bytes)).convert('RGB')
    return np.array(image)

# YOLOv5로 객체 탐지
def detect_objects(model, image):
    results = model(image)
    return results.pandas().xyxy[0]

# 객체 영역 제거 및 배경 채우기 함수
def remove_objects_and_fill_background(image, detections):
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255
    for _, row in detections.iterrows():
        x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
        mask[y1:y2, x1:x2] = 0
    
    # 최빈색 계산
    filled_image = image.copy()
    dominant_color = get_most_frequent_color(image)
    
    # 마스크를 사용하여 최빈색으로 채우기
    filled_image[mask == 0] = dominant_color
    
    return filled_image

# 배경 색상 추출 함수
def extract_background_color(image, background_mask):
    if background_mask is None:
        raise ValueError("배경 마스크가 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_pixels = image[background_mask]
    
    if len(bg_pixels) == 0:
        raise ValueError("배경 픽셀이 없습니다. 배경 색상을 추출할 수 없습니다.")
    
    bg_color = np.mean(bg_pixels, axis=0, dtype=int)
    return bg_color

# 주요 색상 찾기 함수 (KMeans 사용)
def find_dominant_color(image_rgb, k=10):
    pixels = image_rgb.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
    kmeans.fit(pixels)
    centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    label_counts = np.bincount(labels)
    dominant_color = centers[np.argmax(label_counts)]
    return dominant_color.astype(int)
    
# 주요 색상을 시각적으로 확인하는 함수
def plot_colors(centers, ratios):
    patches = np.zeros((50, 300, 3), dtype=int)
    start = 0
    for center, ratio in zip(centers, ratios):
        end = start + int(300 * ratio)
        patches[:, start:end, :] = center
        start = end
    return patches

# RGB에서 HSV로 변환하는 함수
def convert_rgb_to_hsv(rgb_color):
    rgb_color = np.uint8([[rgb_color]])
    hsv_color = cv2.cvtColor(rgb_color, cv2.COLOR_RGB2HSV)
    return hsv_color[0][0]

# HSV 값을 0~360, 0~100, 0~100 범위로 변환하는 함수
def normalize_hsv(hsv):
    h, s, v = hsv
    h = h * 2
    s = s / 255 * 100
    v = v / 255 * 100
    return [h, s, v]

# 톤 결정 함수
def determine_tone(hsv):
    tone_classes = {
        '비비드': (75, 75),
        '스트롱': (60, 67.5),
        '딥': (50, 37.5),
        '라이트': (37.5, 87.5),
        '소프트': (37.5, 62.5),
        '페일': (12.5, 87.5),
        '다크': (62.5, 12.5),
        '덜': (12.5, 37.5)
    }
    
    h, s, v = hsv
    min_distance = float('inf')
    closest_tone = '기타'
    
    for tone, (s_mean, v_mean) in tone_classes.items():
        distance = np.sqrt((s - s_mean)**2 + (v - v_mean)**2)
        if distance < min_distance:
            min_distance = distance
            closest_tone = tone
    
    return closest_tone

# 최빈색 계산
def get_most_frequent_color(image):
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=1, n_init=10, random_state=0)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_.astype(int)[0]
    return dominant_color
    
#########################################################################################
#                         URL Mapping & 서버 실행될 때 함수                               #  
#########################################################################################

@app.route("/flaskServer", methods=["POST"])
@cross_origin(origins='http://localhost:8085', supports_credentials=True)  # 클라이언트의 도메인에 맞게 origins 수정
def flaskServer():
    print("서버 요청 들어옴")
    if 'imageFile' not in request.files:
        return jsonify({'error': '파일이 없습니다'}), 400

    file = request.files['imageFile']
    if file.filename == '':
        return jsonify({'error': '파일 이름이 없습니다'}), 400

    if file:
        image_bytes = file.read()
        image = load_image_from_bytes(image_bytes)
    
        model = load_model()
        detections = detect_objects(model, image)
    
        filled_image = remove_objects_and_fill_background(image, detections)
    
        dominant_color_rgb = find_dominant_color(filled_image)
        dominant_color_hsv = convert_rgb_to_hsv(dominant_color_rgb)
        normalized_dominant_color_hsv = normalize_hsv(dominant_color_hsv)
        color = classify_color_by_hsv(normalized_dominant_color_hsv)
        tone = determine_tone(normalized_dominant_color_hsv)

        print(f'색상: {color}, 톤: {tone}')
    
        #########################################################################################
        #                       세션 정보 불러오기 & 저장 / 톰캣한테 보내기                      #  
        #########################################################################################
            
        loginUserJson = request.form.get('loginUserJson')
        app.logger.debug(f'Received loginUserJson: {loginUserJson}')  # 디버그 로그에 출력
        
        if loginUserJson:
            try:
                user = json.loads(loginUserJson)
                session['login_user'] = user  # 세션에 로그인한 사용자의 userDTO 저장

                # 톰캣 서버(servlet)로 파일 전송
                files = {'file': (file.filename, image_bytes)}
                data = {'color': color, 'tone' : tone, 'login_user': json.dumps(user)}
                response = requests.post(TOMCAT_UPLOAD_URL, files=files, data=data)
                
                if response.status_code != 200:
                    return jsonify({"error": "Failed to send data to Tomcat server"}), 500
                
            except json.JSONDecodeError as e:
                print(f'JSONDecodeError: {e}')  # print 사용
                return jsonify({"error": "Invalid JSON format"}), 400
        else:
            return jsonify({"error": "No userDTO provided"}), 400

        #########################################################################################
        #                                ajax 응답 데이터                                        #  
        #########################################################################################
        
        return jsonify(response.json()), 200

    return jsonify({'error': '파일을 처리하는 중 오류가 발생했습니다'}), 500

#########################################################################################
#                                         서버실행                                       #  
#########################################################################################
if __name__ == "__main__":
    app.run(host='192.168.219.200', port=5058, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://192.168.219.200:5058
Press CTRL+C to quit
